# 🧪 Test de Détection de Pose - SmartFit Coach

Ce notebook permet de tester et expérimenter avec la détection de pose en temps réel.

## Objectifs
- Tester la capture vidéo depuis la webcam
- Vérifier la détection des 33 keypoints avec MediaPipe
- Visualiser le squelette en temps réel
- Analyser les performances (FPS)
- Extraire et sauvegarder les coordonnées des keypoints

## 1. Imports et Configuration

In [3]:
import sys
import os

# Ajouter le dossier parent au path
sys.path.append(os.path.abspath('..'))

import cv2
import numpy as np
import time
import json
from datetime import datetime

from src.detection.video_capture import VideoCapture
from src.detection.pose_detector import PoseDetector
from src.utils.visualization import create_overlay, draw_skeleton

print("✅ Imports réussis")

✅ Imports réussis


## 2. Test de la Capture Vidéo

In [4]:
# Initialisation de la capture vidéo
video_capture = VideoCapture(camera_id=0)

if video_capture.start():
    print("✅ Caméra connectée avec succès")
    width, height = video_capture.get_frame_dimensions()
    fps = video_capture.get_fps()
    print(f"📐 Résolution: {width}x{height}")
    print(f"🎬 FPS configuré: {fps}")
    
    # Test de lecture d'une frame
    success, frame = video_capture.read_frame()
    if success:
        print(f"✅ Frame capturée: {frame.shape}")
    else:
        print("❌ Erreur de capture")
    
    video_capture.release()
else:
    print("❌ Impossible d'accéder à la caméra")

✅ Caméra connectée avec succès
📐 Résolution: 640x480
🎬 FPS configuré: 30.0
✅ Frame capturée: (480, 640, 3)


## 3. Initialisation du Détecteur de Pose

In [5]:
# Créer le détecteur
pose_detector = PoseDetector(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("✅ Détecteur de pose initialisé")
print(f"📊 Nombre de keypoints: {len(PoseDetector.KEYPOINT_NAMES)}")
print(f"🔍 Keypoints disponibles: {PoseDetector.KEYPOINT_NAMES[:10]}...")

✅ Détecteur de pose initialisé
📊 Nombre de keypoints: 33
🔍 Keypoints disponibles: ['nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left']...


## 4. Test de Détection sur une Frame Statique

In [11]:
# Capturer une frame de test
video_capture = VideoCapture(camera_id=0)
video_capture.start()

# Attendre que la caméra se stabilise
for _ in range(10):
    video_capture.read_frame()

success, test_frame = video_capture.read_frame()

if success:
    # Détection
    keypoints = pose_detector.detect(test_frame)
    
    if keypoints:
        print(f"✅ Détection réussie: {len(keypoints)} keypoints trouvés")
        
        # Afficher quelques keypoints
        print("\n📍 Exemples de keypoints:")
        for i in range(min(5, len(keypoints))):
            kp = keypoints[i]
            print(f"  {kp.name}: x={kp.x:.3f}, y={kp.y:.3f}, visibility={kp.visibility:.3f}")
        
        # Dessiner le squelette
        frame_with_skeleton = draw_skeleton(test_frame, keypoints)
        
        # Sauvegarder l'image (optionnel)
        cv2.imwrite('../data/test_detection.jpg', frame_with_skeleton)
        print("\n💾 Image sauvegardée: data/test_detection.jpg")
    else:
        print("⚠️ Aucune personne détectée dans la frame")

video_capture.release()

✅ Détection réussie: 33 keypoints trouvés

📍 Exemples de keypoints:
  nose: x=0.600, y=0.804, visibility=0.999
  left_eye_inner: x=0.634, y=0.713, visibility=0.997
  left_eye: x=0.642, y=0.701, visibility=0.998
  left_eye_outer: x=0.657, y=0.689, visibility=0.996
  right_eye_inner: x=0.554, y=0.717, visibility=0.999

💾 Image sauvegardée: data/test_detection.jpg


## 5. Détection en Temps Réel (Court Test)

In [12]:
# Test de détection en temps réel pendant 5 secondes
video_capture = VideoCapture(camera_id=0)
video_capture.start()

print("🎬 Démarrage de la détection en temps réel (5 secondes)...")
print("Assurez-vous d'être visible par la caméra\n")

start_time = time.time()
frame_count = 0
detection_count = 0
fps_values = []

while time.time() - start_time < 5.0:
    frame_start = time.time()
    
    success, frame = video_capture.read_frame()
    if not success:
        break
    
    # Détection
    keypoints = pose_detector.detect(frame)
    
    if keypoints:
        detection_count += 1
    
    frame_count += 1
    
    # Calcul FPS
    frame_time = time.time() - frame_start
    if frame_time > 0:
        fps_values.append(1.0 / frame_time)

video_capture.release()
pose_detector.release()

# Statistiques
print("\n📊 Résultats:")
print(f"  Frames totales: {frame_count}")
print(f"  Détections réussies: {detection_count} ({detection_count/frame_count*100:.1f}%)")
print(f"  FPS moyen: {np.mean(fps_values):.1f}")
print(f"  FPS min: {np.min(fps_values):.1f}")
print(f"  FPS max: {np.max(fps_values):.1f}")

🎬 Démarrage de la détection en temps réel (5 secondes)...
Assurez-vous d'être visible par la caméra


📊 Résultats:
  Frames totales: 137
  Détections réussies: 137 (100.0%)
  FPS moyen: 30.9
  FPS min: 2.0
  FPS max: 41.7


## 6. Export des Données Keypoints

In [ ]:
# Capturer et exporter les keypoints d'une frame
video_capture = VideoCapture(camera_id=0)
pose_detector_new = PoseDetector()
video_capture.start()

# Capturer
for _ in range(10):
    video_capture.read_frame()

success, frame = video_capture.read_frame()

if success:
    keypoints = pose_detector_new.detect(frame)
    
    if keypoints:
        # Convertir en dictionnaire
        data = pose_detector_new.keypoints_to_dict(keypoints)
        data['timestamp'] = datetime.now().isoformat()
        data['frame_shape'] = frame.shape
        
        # Sauvegarder en JSON
        output_path = '../data/keypoints_sample.json'
        with open(output_path, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"✅ Keypoints exportés vers: {output_path}")
        print(f"📦 Nombre de keypoints: {len(keypoints)}")
    else:
        print("⚠️ Aucune détection pour l'export")

video_capture.release()
pose_detector_new.release()

## 7. Analyse des Keypoints Spécifiques

In [ ]:
# Charger les keypoints sauvegardés
try:
    with open('../data/keypoints_sample.json', 'r') as f:
        data = json.load(f)
    
    keypoints_data = data['keypoints']
    
    print("🔍 Analyse des keypoints importants pour les exercices:\n")
    
    # Points clés pour les squats
    squat_points = ['left_hip', 'right_hip', 'left_knee', 'right_knee', 
                   'left_ankle', 'right_ankle']
    
    print("🏋️ Points pour les SQUATS:")
    for kp in keypoints_data:
        if kp['name'] in squat_points:
            print(f"  {kp['name']:20s} - visibility: {kp['visibility']:.2f}")
    
    print("\n💪 Points pour les POMPES:")
    # Points clés pour les pompes
    pushup_points = ['left_shoulder', 'right_shoulder', 'left_elbow', 
                    'right_elbow', 'left_wrist', 'right_wrist']
    
    for kp in keypoints_data:
        if kp['name'] in pushup_points:
            print(f"  {kp['name']:20s} - visibility: {kp['visibility']:.2f}")
    
except FileNotFoundError:
    print("⚠️ Fichier keypoints_sample.json non trouvé. Exécutez d'abord la cellule 6.")